## 1. libraries

In [2]:
import torch
# import torch.neuron
import torch_neuron

import torchvision
from torchvision import transforms as T
from PIL import Image
import os
from tqdm import tqdm
import time
preprocess_parseq = T.Compose([
            T.Resize((32, 128), T.InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(0.5, 0.5)
        ])

In [3]:
def main_batch(batch_size=8, device_type="cpu", img_folder="/home/ubuntu/parseq/digits_demo", img_type="jpg"):
    if device_type == "cpu":
        device = torch.device("cpu")
        parseq = torch.hub.load('baudm/parseq', 'parseq', pretrained=True).eval()
    elif device_type == "cuda":
        device = torch.device("cuda:0")
        parseq = torch.hub.load('baudm/parseq', 'parseq', pretrained=True).eval().cuda()

    test_img_path = "demo_images/art-01107.jpg"
    img = Image.open(test_img_path).convert('RGB')
    # Preprocess. Model expects a batch of images with shape: (B, C, H, W)
    img = preprocess_parseq(img.convert('RGB')).unsqueeze(0).to(device)

    logits = parseq(img)
    logits.shape  # torch.Size([1, 26, 95]), 94 characters + [EOS] symbol

    # Greedy decoding
    pred = logits.softmax(-1)
    label, confidence = parseq.tokenizer.decode(pred)
    print(confidence)
    print('Decoded label = {}'.format(label[0]))

    dummy_input = torch.rand(batch_size, 3, 32, 128) 

    output_path = "parseq_{}_b{}_torchscript.pt".format(device_type, batch_size)

    # To ONNX
    parseq.to_torchscript(file_path=output_path, method="script", example_inputs=dummy_input)  # opset v14 or newer is required
    
    ts_model = torch.jit.load(output_path)
    logits = ts_model(img)
    logits.shape  # torch.Size([1, 26, 95]), 94 characters + [EOS] symbol
    print(logits.shape)
    # Greedy decoding
    pred = logits.softmax(-1)
    label, confidence = parseq.tokenizer.decode(pred)
    print('Decoded label = {}'.format(label[0]))
    c = 0
    x = 0
    if os.path.exists(img_folder):
        for filename in tqdm(os.listdir(img_folder)):
            if filename.endswith(img_type):
                x += 1
                img_path = os.path.join(img_folder, filename)
                # print(img_path)
                
                img_input = Image.open(img_path).convert('RGB')
                # Preprocess. Model expects a batch of images with shape: (B, C, H, W)
                img_input = preprocess_parseq(img_input.convert('RGB')).unsqueeze(0).to(device)
                # print(img_input)
                with torch.no_grad():  
                    logits_ts = ts_model(img_input)
                    logits = parseq(img_input)

                pred = logits_ts.softmax(-1)
                label_ts, confidence = parseq.tokenizer.decode(pred)
                # print(confidence)
                # print('Decoded label = {}'.format(label_ts[0]))
                
                pred = logits.softmax(-1)
                label, confidence = parseq.tokenizer.decode(pred)
                # print(confidence)
                # print('Decoded label = {}'.format(label[0]))
                
                if label[0] == label_ts[0]:
                    # print("matched")
                    c += 1
                else:
                    print(label[0])
                    print(label_ts[0])
                    # c += 1
    print(c, x)


In [4]:
main_batch(batch_size=4)

Using cache found in /home/ubuntu/.cache/torch/hub/baudm_parseq_main


[tensor([0.9979, 0.9996, 0.9999, 0.9998, 0.9869, 0.9998, 0.9439, 0.9768, 0.9938,
        0.9981], grad_fn=<SliceBackward0>)]
Decoded label = CHEWBACCA


/home/ubuntu/.local/lib/python3.6/site-packages/torch/jit/_recursive.py:235: UserWarning: 'norm' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))
/home/ubuntu/.local/lib/python3.6/site-packages/torch/jit/_recursive.py:614: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  item = getattr(mod, name, None)


torch.Size([1, 26, 95])
Decoded label = CHEWBACCA


100%|██████████| 73/73 [00:10<00:00,  7.11it/s]

73 73


In [10]:
batch_size = 4
parseq = torch.hub.load('baudm/parseq', 'parseq', pretrained=True).eval()

output_path = "parseq_{}_b{}_torchscript.pt".format("cpu", batch_size)
input_holder = torch.rand([1, 3, 32, 128])


ts_model = parseq.to_torchscript(method="trace", example_inputs=input_holder)

allowed_ops = set(torch.neuron.get_supported_operations())
# allowed_ops.remove("aten::view")

neuron_model = torch.neuron.trace(ts_model, example_inputs=input_holder, op_whitelist=allowed_ops)
torch.jit.save(neuron_model,'parseq_model_traced_neuron_new.pt')

Using cache found in /home/ubuntu/.cache/torch/hub/baudm_parseq_main
/home/ubuntu/.local/lib/python3.6/site-packages/torch/jit/_recursive.py:235: UserWarning: 'norm' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  " but it is a non-constant {}. Consider removing it.".format(name, hint))


NotImplementedError: Could not find a torch implementation for prim::If#14

In [8]:

# model_parallel = torch.neuron.DataParallel(neuron_model)

img_folder = "/home/ubuntu/parseq/digits_demo"

input_holder = torch.zeros(batch_size, 3, 32, 128)

img_paths = [os.path.join(img_folder, x) for x in os.listdir(img_folder) if x.endswith("jpg")]

N = len(img_paths)
n_batch = N//batch_size + 1

for i in range(n_batch):
    # print(list(range((i-1)*batch_size,batch_size*i)))
    if batch_size*(i+1) < N:
        input_holder = torch.zeros(batch_size, 3, 32, 128)
        img_paths_tmp = img_paths[(i)*batch_size:batch_size*(i+1)]
    else:
        input_holder = torch.zeros(N-(n_batch-1)*batch_size, 3, 32, 128)
        img_paths_tmp = img_paths[(i)*batch_size:N]
    
    print(input_holder.size())
    for j in range(len(img_paths_tmp)):
                
        img_input = Image.open(img_paths_tmp[j]).convert('RGB')
        img_input = preprocess_parseq(img_input.convert('RGB'))
        # Preprocess. Model expects a batch of images with shape: (B, C, H, W)
        
        input_holder[j, :, :, :] = img_input[:,:,:]
        
    start = time.time()
    logits_ts = neuron_model(input_holder)
    end_1 = time.time()
    logits = parseq(input_holder)
    end_2 = time.time()

    # print("parseq")
    # print(end_2-end_1)
    # print("torchscript")
    # print(end_1-start)
    pred = logits.softmax(-1)
    label, confidence = parseq.tokenizer.decode(pred)
        
    pred = logits_ts.softmax(-1)
    label_ts, confidence = parseq.tokenizer.decode(pred)
    
    print(label_ts, label)



torch.Size([4, 3, 32, 128])


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
/home/ubuntu/.local/lib/python3.6/site-packages/torch_neuron/decorators.py(381): forward
/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/modules/module.py(1090): _slow_forward
/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/modules/module.py(1102): _call_impl
/home/ubuntu/.local/lib/python3.6/site-packages/torch_neuron/graph.py(580): __call__
/home/ubuntu/.local/lib/python3.6/site-packages/torch_neuron/graph.py(209): run_op
/home/ubuntu/.local/lib/python3.6/site-packages/torch_neuron/graph.py(198): __call__
/home/ubuntu/.local/lib/python3.6/site-packages/torch_neuron/runtime.py(69): forward
/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/modules/module.py(1090): _slow_forward
/home/ubuntu/.local/lib/python3.6/site-packages/torch/nn/modules/module.py(1102): _call_impl
/home/ubuntu/.local/lib/python3.6/site-packages/torch/jit/_trace.py(

In [46]:
len(img_paths)

73